# Workflow for Visualizing Esiimators

We can build a simple pipeline for classification:

In [1]:
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.svm import LinearSVC
from sklearn.ensemble import StackingClassifier
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import FeatureUnion
from sklearn.decomposition import TruncatedSVD

In [2]:
pca = PCA(n_components=10)
pca

PCA(copy=True, iterated_power='auto', n_components=10, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

In [3]:
grid = GridSearchCV(pca, {'n_components': [1, 2]})
grid

GridSearchCV(cv=None, error_score=nan,
             estimator=PCA(copy=True, iterated_power='auto', n_components=10,
                           random_state=None, svd_solver='auto', tol=0.0,
                           whiten=False),
             iid='deprecated', n_jobs=None, param_grid={'n_components': [1, 2]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [4]:
pca = PCA()
lr = LogisticRegression()
pipe1 = Pipeline([
    ('pca', PCA()),
    ('lr', LogisticRegression())
])

pipe1

Pipeline(memory=None,
         steps=[('pca',
                 PCA(copy=True, iterated_power='auto', n_components=None,
                     random_state=None, svd_solver='auto', tol=0.0,
                     whiten=False)),
                ('lr',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=1,
                                    l1_ratio=None, max_iter=100,
                                    multi_class='auto', n_jobs=None,
                                    penalty='l2', random_state=None,
                                    solver='lbfgs', tol=0.0001, verbose=0,
                                    warm_start=False))],
         verbose=False)

To add a column transformer that handles categories and numerical features:

In [5]:
cat_pipe = Pipeline([
    ('impute', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

num_pipe = Pipeline([
    ('impute', SimpleImputer()),
    ('scale', StandardScaler())
])

preprcoessor = ColumnTransformer([
    ('cat', cat_pipe, ['gender', 'age_bracket']),
    ('num', num_pipe, ['age', 'income'])
])

preprcoessor

ColumnTransformer(n_jobs=None, remainder='drop', sparse_threshold=0.3,
                  transformer_weights=None,
                  transformers=[('cat',
                                 Pipeline(memory=None,
                                          steps=[('impute',
                                                  SimpleImputer(add_indicator=False,
                                                                copy=True,
                                                                fill_value='missing',
                                                                missing_values=nan,
                                                                strategy='constant',
                                                                verbose=0)),
                                                 ('onehot',
                                                  OneHotEncoder(categories='auto',
                                                                drop=None,
                                                                dtype=<class 'numpy.float64'>,
                                                                handle_unknown='ignore',
                                                                sparse=True))],
                                          verbose=False),
                                 ['gender', 'age_bracket']),
                                ('num',
                                 Pipeline(memory=None,
                                          steps=[('impute',
                                                  SimpleImputer(add_indicator=False,
                                                                copy=True,
                                                                fill_value=None,
                                                                missing_values=nan,
                                                                strategy='mean',
                                                                verbose=0)),
                                                 ('scale',
                                                  StandardScaler(copy=True,
                                                                 with_mean=True,
                                                                 with_std=True))],
                                          verbose=False),
                                 ['age', 'income'])],
                  verbose=False)

Next, we can attached a classifier to this preprocessor:

In [6]:
pipe2 = Pipeline([
    ('preprocessor', preprcoessor),
    ('lr', lr)
])

pipe2

Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('cat',
                                                  Pipeline(memory=None,
                                                           steps=[('impute',
                                                                   SimpleImputer(add_indicator=False,
                                                                                 copy=True,
                                                                                 fill_value='missing',
                                                                                 missing_values=nan,
                                                                                 strategy='constant',
                                                                                 verbose=0)),
                                                                  ('onehot',
                                                                   OneHotEncoder(categories='au...
                                                                   StandardScaler(copy=True,
                                                                                  with_mean=True,
                                                                                  with_std=True))],
                                                           verbose=False),
                                                  ['age', 'income'])],
                                   verbose=False)),
                ('lr',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=1,
                                    l1_ratio=None, max_iter=100,
                                    multi_class='auto', n_jobs=None,
                                    penalty='l2', random_state=None,
                                    solver='lbfgs', tol=0.0001, verbose=0,
                                    warm_start=False))],
         verbose=False)

We can build a more complicated classifier at the end:

In [7]:
vote_clf = VotingClassifier([
    ('rf', RandomForestClassifier()),
    ('mlp', MLPClassifier()),
    ('lr', LogisticRegression())
])

pipe3  = Pipeline([
    ('preprocessor', preprcoessor),
    ('clf', vote_clf)
])

pipe3

Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('cat',
                                                  Pipeline(memory=None,
                                                           steps=[('impute',
                                                                   SimpleImputer(add_indicator=False,
                                                                                 copy=True,
                                                                                 fill_value='missing',
                                                                                 missing_values=nan,
                                                                                 strategy='constant',
                                                                                 verbose=0)),
                                                                  ('onehot',
                                                                   OneHotEncoder(categories='au...
                                                             warm_start=False)),
                                              ('lr',
                                               LogisticRegression(C=1.0,
                                                                  class_weight=None,
                                                                  dual=False,
                                                                  fit_intercept=True,
                                                                  intercept_scaling=1,
                                                                  l1_ratio=None,
                                                                  max_iter=100,
                                                                  multi_class='auto',
                                                                  n_jobs=None,
                                                                  penalty='l2',
                                                                  random_state=None,
                                                                  solver='lbfgs',
                                                                  tol=0.0001,
                                                                  verbose=0,
                                                                  warm_start=False))],
                                  flatten_transform=True, n_jobs=None,
                                  verbose=False, voting='hard',
                                  weights=None))],
         verbose=False)

Next, we can do some dimensional reduction with a PCA and a Truncated SVD:

In [8]:
dim_reduce = FeatureUnion([("pca", PCA(n_components=1)),
                           ("svd", TruncatedSVD(n_components=2))])

pipe4 = Pipeline([
    ('preprocessor', preprcoessor),
    ('dim_reduce', dim_reduce),
    ('clf', vote_clf)
])

pipe4

Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('cat',
                                                  Pipeline(memory=None,
                                                           steps=[('impute',
                                                                   SimpleImputer(add_indicator=False,
                                                                                 copy=True,
                                                                                 fill_value='missing',
                                                                                 missing_values=nan,
                                                                                 strategy='constant',
                                                                                 verbose=0)),
                                                                  ('onehot',
                                                                   OneHotEncoder(categories='au...
                                                             warm_start=False)),
                                              ('lr',
                                               LogisticRegression(C=1.0,
                                                                  class_weight=None,
                                                                  dual=False,
                                                                  fit_intercept=True,
                                                                  intercept_scaling=1,
                                                                  l1_ratio=None,
                                                                  max_iter=100,
                                                                  multi_class='auto',
                                                                  n_jobs=None,
                                                                  penalty='l2',
                                                                  random_state=None,
                                                                  solver='lbfgs',
                                                                  tol=0.0001,
                                                                  verbose=0,
                                                                  warm_start=False))],
                                  flatten_transform=True, n_jobs=None,
                                  verbose=False, voting='hard',
                                  weights=None))],
         verbose=False)

In [10]:
# Single PCA for dimensino reduction
pipe4.set_params(dim_reduce=PCA())

Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('cat',
                                                  Pipeline(memory=None,
                                                           steps=[('impute',
                                                                   SimpleImputer(add_indicator=False,
                                                                                 copy=True,
                                                                                 fill_value='missing',
                                                                                 missing_values=nan,
                                                                                 strategy='constant',
                                                                                 verbose=0)),
                                                                  ('onehot',
                                                                   OneHotEncoder(categories='au...
                                                             warm_start=False)),
                                              ('lr',
                                               LogisticRegression(C=1.0,
                                                                  class_weight=None,
                                                                  dual=False,
                                                                  fit_intercept=True,
                                                                  intercept_scaling=1,
                                                                  l1_ratio=None,
                                                                  max_iter=100,
                                                                  multi_class='auto',
                                                                  n_jobs=None,
                                                                  penalty='l2',
                                                                  random_state=None,
                                                                  solver='lbfgs',
                                                                  tol=0.0001,
                                                                  verbose=0,
                                                                  warm_start=False))],
                                  flatten_transform=True, n_jobs=None,
                                  verbose=False, voting='hard',
                                  weights=None))],
         verbose=False)

In [11]:
estimators = [('lr', LogisticRegression()), ('svc', LinearSVC())]
clf = StackingClassifier(
    estimators=estimators, final_estimator=None)
clf

StackingClassifier(cv=None,
                   estimators=[('lr',
                                LogisticRegression(C=1.0, class_weight=None,
                                                   dual=False,
                                                   fit_intercept=True,
                                                   intercept_scaling=1,
                                                   l1_ratio=None, max_iter=100,
                                                   multi_class='auto',
                                                   n_jobs=None, penalty='l2',
                                                   random_state=None,
                                                   solver='lbfgs', tol=0.0001,
                                                   verbose=0,
                                                   warm_start=False)),
                               ('svc',
                                LinearSVC(C=1.0, class_weight=None, dual=True,
                                          fit_intercept=True,
                                          intercept_scaling=1,
                                          loss='squared_hinge', max_iter=1000,
                                          multi_class='ovr', penalty='l2',
                                          random_state=None, tol=0.0001,
                                          verbose=0))],
                   final_estimator=None, n_jobs=None, passthrough=False,
                   stack_method='auto', verbose=0)

In [12]:
pipe5 = Pipeline([
    ('preprocessor', preprcoessor),
    ('dim_reduce', dim_reduce),
    ('hello', PCA()),
    ('stacker', clf)
])
pipe5

Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('cat',
                                                  Pipeline(memory=None,
                                                           steps=[('impute',
                                                                   SimpleImputer(add_indicator=False,
                                                                                 copy=True,
                                                                                 fill_value='missing',
                                                                                 missing_values=nan,
                                                                                 strategy='constant',
                                                                                 verbose=0)),
                                                                  ('onehot',
                                                                   OneHotEncoder(categories='au...
                                                                    solver='lbfgs',
                                                                    tol=0.0001,
                                                                    verbose=0,
                                                                    warm_start=False)),
                                                ('svc',
                                                 LinearSVC(C=1.0,
                                                           class_weight=None,
                                                           dual=True,
                                                           fit_intercept=True,
                                                           intercept_scaling=1,
                                                           loss='squared_hinge',
                                                           max_iter=1000,
                                                           multi_class='ovr',
                                                           penalty='l2',
                                                           random_state=None,
                                                           tol=0.0001,
                                                           verbose=0))],
                                    final_estimator=None, n_jobs=None,
                                    passthrough=False, stack_method='auto',
                                    verbose=0))],
         verbose=False)

In [13]:
pipe5._re

AttributeError: 'Pipeline' object has no attribute '_re'

In [ ]:
# meta estimator
GridSearchCV(pipe5, {})

In [14]:
# weird pipeline
cat_pipe2 = Pipeline([
    ('impute', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

num_pipe = Pipeline([
    ('impute', SimpleImputer()),
    ('scale', StandardScaler()),
    ('dim_reduce', dim_reduce)
])

preprcoessor = ColumnTransformer([
    ('cat', cat_pipe2, ['gender', 'age_bracket']),
    ('num', num_pipe, ['age', 'income'])
])

pipe6 = Pipeline([
    ('prep', preprcoessor),
    ('clf', vote_clf)
])
pipe6

Pipeline(memory=None,
         steps=[('prep',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('cat',
                                                  Pipeline(memory=None,
                                                           steps=[('impute',
                                                                   SimpleImputer(add_indicator=False,
                                                                                 copy=True,
                                                                                 fill_value='missing',
                                                                                 missing_values=nan,
                                                                                 strategy='constant',
                                                                                 verbose=0)),
                                                                  ('onehot',
                                                                   OneHotEncoder(categories='auto',
                                                                                 drop...
                                                             warm_start=False)),
                                              ('lr',
                                               LogisticRegression(C=1.0,
                                                                  class_weight=None,
                                                                  dual=False,
                                                                  fit_intercept=True,
                                                                  intercept_scaling=1,
                                                                  l1_ratio=None,
                                                                  max_iter=100,
                                                                  multi_class='auto',
                                                                  n_jobs=None,
                                                                  penalty='l2',
                                                                  random_state=None,
                                                                  solver='lbfgs',
                                                                  tol=0.0001,
                                                                  verbose=0,
                                                                  warm_start=False))],
                                  flatten_transform=True, n_jobs=None,
                                  verbose=False, voting='hard',
                                  weights=None))],
         verbose=False)